In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-03-31 12:40:53--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-03-31 12:40:55 (1.05 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Homework_week_22").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [0]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
# url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
url="https://amazon-review-bucket.s3.amazonaws.com/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

user_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep='\t', header=True)

# Show DataFrame
user_data_df.show()
user_data_df.count()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

2643619

In [0]:
# Drop null values
user_data_df = user_data_df.dropna()
user_data_df.count()

2643241

In [0]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with verified purchases, a minimum of 1 helpful vote and a minimum of 5 total
cleaned_df = user_data_df.filter(col("verified_purchase")  == "Y")
cleaned_df = cleaned_df.filter(col("helpful_votes")  > 0)
cleaned_df = cleaned_df.filter(col("total_votes")  > 5)
print("Total Number of Pet Product Reviews through Vine: ", cleaned_df.count())
cleaned_df.show()

Total Number of Pet Product Reviews through Vine:  117813
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    6454837|R2D06CYY6KZSJ0|B00FEEFM6Y|     314480388|Zuke’s Skinny Bak...|    Pet Products|          2|           16|         19|   N|                Y|        Not the best|My dog doesn't li...| 2015-08-31|
|         US|    1820095|R2P2G1XUTS2UI1|B0002565N4|     419360

In [0]:
# Create user dataframe to find count of reviews per customer (for table Customers )
df_customers = cleaned_df.groupby('customer_id').agg({'review_id': 'count'})
df_customers = df_customers.withColumnRenamed('count(review_id)','customer_count')
print("Total number of Customers: ", df_customers.count())
df_customers.show()

Total number of Customers:  103016
+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   10250915|             1|
|   38830324|             1|
|   14505313|             1|
|   20850834|             1|
|   46565651|             1|
|   46836847|             1|
|   51005012|             1|
|   44169385|             1|
|   46990092|             2|
|   10252330|             1|
|   14348125|             1|
|   50878423|             1|
|   10654670|             1|
|   42014357|             1|
|   39887185|             1|
|   13261276|             1|
|   20133027|             1|
|   35869248|             1|
|     785286|             1|
|   50003658|             1|
+-----------+--------------+
only showing top 20 rows



In [0]:
df_customers = df_customers.selectExpr("cast(customer_id as int) customer_id","cast(customer_count as int) customer_count")
df_customers.dtypes

[('customer_id', 'int'), ('customer_count', 'int')]

In [0]:
# Create user dataframe to store product Information
df_products = cleaned_df.select(['Product_ID','product_title','product_category']).distinct()
print("Total number of Products: ", df_products.count())
df_products.show()

Total number of Products:  37234
+----------+--------------------+----------------+
|Product_ID|       product_title|product_category|
+----------+--------------------+----------------+
|B00V74ZREW|Wondercide Natura...|    Pet Products|
|B0117ZVPE6|Tetra ReptoHeat B...|    Pet Products|
|B00M0PC37G|Liteaid Wood Adju...|    Pet Products|
|B01123MDIK|Merax Pet Strolle...|    Pet Products|
|B00PGKWGMY|Service Dog Harne...|    Pet Products|
|B005FU4Z8I|Kaytee Hay n Food...|    Pet Products|
|B003BK9XPY|Big Button Dog Ca...|    Pet Products|
|B000S5JN5K|Esha Gastropex Aq...|    Pet Products|
|B00OP6R094|AmazonBasics Elev...|    Pet Products|
|B00BUFTKBE|Coastal Pet Produ...|    Pet Products|
|B00ZSN5RUW|Fur Family Cat Tr...|    Pet Products|
|B00B9GRQTM|Generic GT-211SW ...|    Pet Products|
|B000H904WS|MidWest Homes for...|    Pet Products|
|B003SLTETG|High Tech Pet Pow...|    Pet Products|
|B00ET0C4WE|ANGELS' EYES Whit...|    Pet Products|
|B0018CE5PA|Aqueon Fern Aquar...|    Pet Products

In [0]:
df_products.dtypes

[('Product_ID', 'string'),
 ('product_title', 'string'),
 ('product_category', 'string')]

In [0]:
# Create user dataframe for review_id_table
df_review_id_table = cleaned_df.select(["review_id", "customer_id","product_id","product_parent","review_date"])
print("Total number of Reviews: ", df_review_id_table.count())
df_review_id_table.show()

Total number of Reviews:  117813
+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2D06CYY6KZSJ0|    6454837|B00FEEFM6Y|     314480388| 2015-08-31|
|R2P2G1XUTS2UI1|    1820095|B0002565N4|     419360744| 2015-08-31|
|R2E9I9L1DXK7U8|   10299380|B00DJZ0SE6|     298598234| 2015-08-31|
| RT0UQXBO40MI5|   13497289|B00KXE0SJ6|     626985166| 2015-08-31|
| RF0AI0E9TBNEK|   16360624|B00KR3PROE|     523188961| 2015-08-31|
|R21KC552Y6HL8X|   51227803|B00E2E1XBK|     180836080| 2015-08-31|
| ROJU3NC7YJOX2|   12317031|B00IUP9COW|     352556125| 2015-08-31|
| RX9WC9FTIR1XR|   52747799|B00DZITYGU|     598776873| 2015-08-31|
|  RTVXISV5QOKQ|   43213184|B00BJB324K|     988247002| 2015-08-31|
|R1ZTQ09ZCNTMDO|    4417771|B007GCET7K|     885271830| 2015-08-31|
| RIRW4EE0K3Y1Y|   15722762|B000H3VAQ8|     518556199| 2015-08-31|
| RGDCOU1KBHMNG|   23340430|B

In [0]:
df_review_id_table = df_review_id_table.selectExpr("cast(review_id as string) review_id","cast(customer_id as int) customer_id","cast(product_id as string) product_id","cast(product_parent as int) product_parent","cast(review_date as date) review_date")
df_review_id_table.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [0]:
df_review_id_table.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2D06CYY6KZSJ0|    6454837|B00FEEFM6Y|     314480388| 2015-08-31|
|R2P2G1XUTS2UI1|    1820095|B0002565N4|     419360744| 2015-08-31|
|R2E9I9L1DXK7U8|   10299380|B00DJZ0SE6|     298598234| 2015-08-31|
| RT0UQXBO40MI5|   13497289|B00KXE0SJ6|     626985166| 2015-08-31|
| RF0AI0E9TBNEK|   16360624|B00KR3PROE|     523188961| 2015-08-31|
|R21KC552Y6HL8X|   51227803|B00E2E1XBK|     180836080| 2015-08-31|
| ROJU3NC7YJOX2|   12317031|B00IUP9COW|     352556125| 2015-08-31|
| RX9WC9FTIR1XR|   52747799|B00DZITYGU|     598776873| 2015-08-31|
|  RTVXISV5QOKQ|   43213184|B00BJB324K|     988247002| 2015-08-31|
|R1ZTQ09ZCNTMDO|    4417771|B007GCET7K|     885271830| 2015-08-31|
| RIRW4EE0K3Y1Y|   15722762|B000H3VAQ8|     518556199| 2015-08-31|
| RGDCOU1KBHMNG|   23340430|B00N9SL8SS|     721942363| 2015-08

In [0]:
# Create user dataframe for vine data 
df_vine_table = cleaned_df.select(["review_id", "star_rating","helpful_votes","total_votes","vine"])
print("Total number of Vine Records: ", df_vine_table.count())
df_vine_table.show()

Total number of Vine Records:  117813
+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R2D06CYY6KZSJ0|          2|           16|         19|   N|
|R2P2G1XUTS2UI1|          5|            8|          8|   N|
|R2E9I9L1DXK7U8|          5|           10|         10|   N|
| RT0UQXBO40MI5|          5|            6|          6|   N|
| RF0AI0E9TBNEK|          3|            6|          6|   N|
|R21KC552Y6HL8X|          1|           27|         31|   N|
| ROJU3NC7YJOX2|          1|            8|          8|   N|
| RX9WC9FTIR1XR|          5|           25|         25|   N|
|  RTVXISV5QOKQ|          2|            6|          6|   N|
|R1ZTQ09ZCNTMDO|          5|           12|         12|   N|
| RIRW4EE0K3Y1Y|          5|            5|          6|   N|
| RGDCOU1KBHMNG|          3|           29|         31|   N|
| RVTYWID2TPMMY|          2|           35|         42|   N|
|R

In [0]:
df_vine_table = df_vine_table.selectExpr("cast(review_id as string) review_id","cast(star_rating as int) star_rating","cast(helpful_votes as int) helpful_votes","cast(total_votes as int) total_votes","cast(vine as string) vine")
df_vine_table.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string')]

Postgres Setup

In [0]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://amazon-review-avr.cn0fs69uwn5x.us-east-1.rds.amazonaws.com:5432/amazon_review_db"
config = {"user":"root", 
          "password": "Welcome!", 
          "driver":"org.postgresql.Driver"}


In [0]:
# Write DataFrame to customer table in RDS

df_customers.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [0]:
# Write dataframe to products table in RDS

df_products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [0]:
# Write dataframe to review_id table in RDS

df_review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [0]:
# Write dataframe to vine table in RDS

df_vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

In [0]:
from pyspark import SparkFiles
from pyspark.sql.functions import col


url="https://amazon-review-bucket.s3.amazonaws.com/t_test.csv"
spark.sparkContext.addFile(url)
df_ttest = spark.read.csv(SparkFiles.get("t_test.csv"), sep=',', header=True)

In [23]:
df_ttest.show()

+---+------+------+------+------+------+
|_c0|1 star|2 star|3 star|4 star|5 star|
+---+------+------+------+------+------+
|  N| 22.56|  7.47|  9.12| 13.76| 47.09|
|  Y| 13.64|  4.55|  4.55| 13.64| 63.64|
+---+------+------+------+------+------+



In [39]:
import scipy
vine_rating = df_ttest.filter(col("_c0")=="Y")
not_vine_rating = df_ttest.filter(col("_c0")=="N")

vine_rating.select(['5 star']).show()
scipy.stats.ttest_ind(vine_rating.select(['5 star']), not_vine_rating.select(['5 star']), equal_var=False)

+------+
|5 star|
+------+
| 63.64|
+------+



/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3584: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)


TypeError: ignored